In [1]:
import xarray as xr
import plotly.express as px
from pathlib import Path
from datetime import datetime

In [2]:
station_name = "CALCOFI"
release_folder = "../../../../release/"

In [3]:
calcofi_data = xr.load_dataset("../../3_post_processed/calcofi_zooplankton.nc")
calcofi_data

<xarray.Dataset> Size: 353MB
Dimensions:      (is_day: 2, time: 7796, latitude: 55, longitude: 103)
Coordinates:
  * is_day       (is_day) bool 2B False True
  * time         (time) datetime64[ns] 62kB 1951-01-09 1951-01-10 ... 2023-01-25
  * latitude     (latitude) float64 440B 0.5 1.5 2.5 3.5 ... 51.5 52.5 53.5 54.5
  * longitude    (longitude) float64 824B -179.5 -178.5 -177.5 ... -78.5 -77.5
Data variables:
    zooplankton  (is_day, time, latitude, longitude) float32 353MB nan ... nan

In [4]:
results = xr.Dataset(
    {
        "day": calcofi_data["zooplankton"].sel(is_day=True).drop_vars("is_day"),
        "night": calcofi_data["zooplankton"].sel(is_day=False).drop_vars("is_day"),
    }
).compute()
results

<xarray.Dataset> Size: 353MB
Dimensions:    (time: 7796, latitude: 55, longitude: 103)
Coordinates:
  * time       (time) datetime64[ns] 62kB 1951-01-09 1951-01-10 ... 2023-01-25
  * latitude   (latitude) float64 440B 0.5 1.5 2.5 3.5 ... 51.5 52.5 53.5 54.5
  * longitude  (longitude) float64 824B -179.5 -178.5 -177.5 ... -78.5 -77.5
Data variables:
    day        (time, latitude, longitude) float32 177MB nan nan nan ... nan nan
    night      (time, latitude, longitude) float32 177MB nan nan nan ... nan nan

In [5]:
day_df = (
    results["day"].mean(["latitude", "longitude"]).to_dataframe().reset_index().dropna()
)
night_df = (
    results["night"]
    .mean(["latitude", "longitude"])
    .to_dataframe()
    .reset_index()
    .dropna()
)


fig = px.scatter(
    title="Day and Night Series",
    labels={"value": "Abundance", "time": "Time"},
    color_discrete_sequence=["blue", "orange"],
)

fig.add_scatter(
    x=day_df["time"],
    y=day_df["day"],
    name="Day",
    mode="markers",
    marker=dict(color="orange", size=4),
)

fig.add_scatter(
    x=night_df["time"],
    y=night_df["night"],
    name="Night",
    mode="markers",
    marker=dict(color="blue", size=4),
)

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Carbon weight mg/m3",
    legend_title="Day/Night",
    legend=dict(orientation="h"),
)
fig.show()

In [6]:
data_export = (
    results.expand_dims({"layer": [0]})
    .transpose("time", "latitude", "longitude", "layer")
    .dropna(dim="time", how="all")
    .dropna(dim="latitude", how="all")
    .dropna(dim="longitude", how="all")
)
data_export["day"].attrs = {
    "description": "Zooplankton carbon weight by square meter of sea water at day",
    "units": "mg/m3",
}
data_export["night"].attrs = {
    "description": "Zooplankton carbon weight by square meter of sea water at night",
    "units": "mg/m3",
}
data_export["layer"].attrs = {
    "flag_values": [0, 1, 2],
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "layer",
    "long_name": "Layer depth",
    "axis": "Z",
}
data_export.attrs = {
    "station": station_name,
    "creation date": datetime.now().isoformat(),
}
data_export

<xarray.Dataset> Size: 353MB
Dimensions:    (layer: 1, time: 7796, latitude: 55, longitude: 103)
Coordinates:
  * layer      (layer) int64 8B 0
  * time       (time) datetime64[ns] 62kB 1951-01-09 1951-01-10 ... 2023-01-25
  * latitude   (latitude) float64 440B 0.5 1.5 2.5 3.5 ... 51.5 52.5 53.5 54.5
  * longitude  (longitude) float64 824B -179.5 -178.5 -177.5 ... -78.5 -77.5
Data variables:
    day        (time, latitude, longitude, layer) float32 177MB nan nan ... nan
    night      (time, latitude, longitude, layer) float32 177MB nan nan ... nan
Attributes:
    station:        CALCOFI
    creation date:  2025-07-09T13:29:28.222363

In [7]:
data_export = data_export.chunk(
    {"time": -1, "latitude": 10, "longitude": 10, "layer": -1}
)

In [8]:
data_export.to_zarr(Path(release_folder) / f"{station_name}_zooplankton.zarr", mode="w")